# Get SocialVec embeddings
This code fetches the SocialVec embeedings for a given list of Twitter users

In [1]:
#config related imports
import numpy as np
import pandas as pd
import tomli
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from gensim.models import Word2Vec
from tqdm import tqdm
from tqdm import trange
import time
import ast

#for logging
import logging
from importlib import reload
import datetime


# Configurations

In [2]:
with open("../config.toml", "rb") as f:
    config = tomli.load(f)
    
# Additional Configs:
limit_amount_of_users = True
users_limitation = 30

#Setup logging
reload(logging)
now = datetime.datetime.now()
now = now.strftime("%Y%m%d_%H%M%S")
logging.basicConfig(filename=f"logs/SocialVec Collector_{now}.log", filemode='w', format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p',level=logging.DEBUG)


## Load SocialVec and list of popular users

In [3]:

SocialVec = Word2Vec.load(config["SocialVec"]["model_path"])
SocialVec_popular_list = SocialVec.wv.index_to_key

loading Word2Vec object from /Users/nlotan/code/phd/SocialVec/SocialVec/models/SocialVec_v6_sg_all.model
loading wv recursively from /Users/nlotan/code/phd/SocialVec/SocialVec/models/SocialVec_v6_sg_all.model.wv.* with mmap=None
loading vectors from /Users/nlotan/code/phd/SocialVec/SocialVec/models/SocialVec_v6_sg_all.model.wv.vectors.npy with mmap=None
setting ignored attribute vectors_norm to None
loading vocabulary recursively from /Users/nlotan/code/phd/SocialVec/SocialVec/models/SocialVec_v6_sg_all.model.vocabulary.* with mmap=None
loading trainables recursively from /Users/nlotan/code/phd/SocialVec/SocialVec/models/SocialVec_v6_sg_all.model.trainables.* with mmap=None
loading syn1neg from /Users/nlotan/code/phd/SocialVec/SocialVec/models/SocialVec_v6_sg_all.model.trainables.syn1neg.npy with mmap=None
setting ignored attribute cum_table to None
Word2Vec lifecycle event {'fname': '/Users/nlotan/code/phd/SocialVec/SocialVec/models/SocialVec_v6_sg_all.model', 'datetime': '2021-12-30T

# Twitter connection
For each relevant user, collet the list of popular users they follow.

In [4]:
# Open Twitter API for the cases that users do not exist in SocialVec yet.
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(config["twitter_api"]["consumer_key"],
                           config["twitter_api"]["consumer_secret"])
  
# set access to user's access key and access secret 
auth.set_access_token(config["twitter_api"]["access_token"], 
                      config["twitter_api"]["access_token_secret"])
  
# calling the api 
api = tweepy.API(auth, wait_on_rate_limit=False,proxy="http://proxy-chain.intel.com:911")

## Auxiliary Function

In [5]:
def twitter_sleep():
    sleep_time = 600 * 15
    for i in trange(sleep_time,
                    desc = f"sleep for {sleep_time/600} minutes"):
        time.sleep(0.1)

## Get list of users from file
We use users limitation to limit the amount of users we collect for explortaions

In [7]:
collect_by = "user_id"

In [14]:
users_df = pd.read_csv('user_tweets.csv',names=["tweet_id","user_id"])
users_df

,tweet_id,user_id
0,921954017885921280,861364736499175424
1,876939282069544960,3434877514
2,791784139447865344,860686237
3,826459912192589824,1194398761
4,904028651183800320,810211113891745792
5,844326831474053120,22682450
6,860515988537823232,568546624
7,881361728885456896,86619378
8,900478823896150016,2873105092
9,838265495879094272,798282754949574660


In [13]:
if collect_by == "screen_name":
    users_df = pd.read_csv('../Data/datasets/twitter-incivility/candidate_tweet_responses.csv')

    #we only need screen name of user_id
    users_df = users_df[['screen_name']].drop_duplicates()

    if limit_amount_of_users:
        users_df = users_df.head(users_limitation)

    #Set index for later required access
    users_df = users_df.set_index('screen_name')
    
if collect_by == "user_id":
    users_df = pd.read_csv('user_tweets.csv',names=["tweet_id","user_id"])

    #we only need screen name of user_id
    users_df = users_df[['user_id']].drop_duplicates()

    if limit_amount_of_users:
        users_df = users_df.head(users_limitation)

    #Set index for later required access
    users_df = users_df.set_index('user_id')

TypeError: unhashable type: 'list'

Backing off send_request(...) for 211.1s (requests.exceptions.ConnectTimeout: HTTPSConnectionPool(host='api.segment.io', port=443): Max retries exceeded with url: /v1/batch (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x172e67a00>, 'Connection to api.segment.io timed out. (connect timeout=15)')))


# Get SocialVec vectors per user
## Get list of friends per user

In [ ]:
for user in users_df.iterrows():
    user = user[0]

    for attempt in range(10):
        try:
            print (f"user#:{user}")
            friends = sorted(api.get_friend_ids(screen_name=user))
            users_df.at[user, 'friends_list'] = str(friends)
            logging.debug(f"Completed user:{user}")
            users_df.at[user, 'friends_available'] = "Yes"

        except tweepy.errors.TweepyException as e:

            if 'Rate limit exceeded' in e.api_messages:
                print(f"Rate limit exceeded. Wait and retry {10-attempt} more times...")
                twitter_sleep()
                continue #retry
            else:
                print (f"error: {e.api_messages}")
                logging.debug(f"user#:{user} Error{e.response}")
                break # we failed not because of rate limit
        # We succeeded
        break
    #completed all attempts

In [ ]:
users_df.to_csv("users_with_friends.csv")

In [ ]:
def get_popular_list( row ):

    followers = ast.literal_eval(row['friends_list'])
    #transform users ids from int list to string list
    item_str = [str(item) for item in followers]
    return str(item_str)

In [ ]:
def friends_to_SocialVec( row ):
    followers = ast.literal_eval(row['friends_list'])
    #transform users ids from int list to string list
    item_str = [str(item) for item in followers]
    
    #keep only popular users in the list:
    popular_list = [i for i in item_str if i in SocialVec_popular_list]
    
    user_embedding = np.mean(SocialVec.wv[popular_list], axis=0)

    return str(user_embedding.tolist())

In [ ]:
df = pd.DataFrame(users_df.iloc[3]).T

In [ ]:
df['temp'] = df.apply(friends_to_SocialVec, axis=1)